In [1]:
import pandas as pd
import numpy as np
import openpyxl as excel
import sys

import xml.etree.ElementTree as et
import os
import glob
import shutil
import subprocess

In [2]:
#加工条件excelファイルのファイル名を定義（同一フォルダー内に置く事）
tecsets_excelfile_name = 'VFHVRB_S.xlsx'

#.xmlを出力する際のフォルダー名
folder_name = 'VFHVRB'

#.xmlをエクスポートする際のファイル名
export_file_name = 'VFHVRB.xml'


In [3]:
#加工条件ファイルを読み込んで結合解除

file = tecsets_excelfile_name #ファイル名を定義
src_book = excel.load_workbook(file, data_only=True) #Excelファイルを読み込み オプション:値のみ
sheet_names = src_book.sheetnames #シートネームをリストで取得
dst_book = excel.Workbook() #空のExcelワークブックを作成

for cnt, sn in enumerate(sheet_names): #複数シートがある場合は全て実行する(今回は必要無し)
    if cnt == 0: #初回のみ空のワークブックに直接書き込む
        dst_sheet = dst_book.active
        dst_sheet.title = sn
    else:
        dst_sheet = dst_book.create_sheet(sn) #２回目以降はシートを作成して書き込む

    src_sheet = src_book[sn]
    merged_ranges = src_sheet.merged_cells.ranges #目標シートの結合セルの左上と右下のセル番号をリストで全て取得

    for row in src_sheet: 
        for cell in row:
            dst_sheet[cell.coordinate].value = cell.value #目標シートをsrc_sheetにコピーする

    for mr in merged_ranges:
        src_val = src_sheet.cell(mr.min_row, mr.min_col).value #左上の値を取得
        for row in range(mr.min_row, mr.max_row + 1): #列を書き込み
            for col in range(mr.min_col, mr.max_col + 1): #行を書き込み
                dst_sheet.cell(row, col).value = src_val

dst_book.save('normalize.xlsx')

In [4]:
#材質リストを作成
mat_file = 'normalize.xlsx'
materials_record = pd.read_excel(mat_file , index_col=None , header=None)#読み込み
materials_record = materials_record.iloc[0]#必要行のみ抽出
materials_list = materials_record.dropna()#NaN値の除去
materials_list = materials_list.unique()#ユニーク値の抽出

In [5]:
#加工条件表ファイルの読み込み
file_name = 'normalize.xlsx'
tecsets_db = pd.read_excel(file_name , index_col=None , header=0, skiprows=[0])

#加工条件ファイルの確認用（分類前）
tecsets_db

In [6]:
#加工条件データフレームを材質毎に分類する
for i, j in enumerate(materials_list):#リストのカウンターとリスト内容で回す
    if i==0:#初回のみそのまま読み込む
        tecsets_mdb = pd.merge(tecsets_db.iloc[:,0:3],tecsets_db.loc[:,'min-1':'ae mm'], left_index=True, right_index=True)#寸法と第一材質を読み込み
        tecsets_mdb.insert(3, '材質', j)#材質カラムを追加
        col = tecsets_mdb.columns#オリジナルのカラム名をリストで取得
    else:#その後は条件の後ろに".1"をつけて材質の境界を識別している
        a = pd.merge(tecsets_db.iloc[:,0:3],tecsets_db.loc[:,'min-1.'+str(i):'ae mm.'+str(i)], left_index=True, right_index=True)#第二材質以降を読み込む
        a.insert(3, '材質', j)#材質カラムを追加
        b = a.columns#現在のカラム名を取得
        dic = dict(zip(b, col))#オリジナルのカラム名と現在のカラム名を変更用に辞書化
        a = a.rename(columns=dic)#カラム名をオリジナルに変更
        tecsets_mdb=tecsets_mdb.append(a, ignore_index=True) #新しいデータフレームを下に結合    


#加工条件ファイルの確認用（分類後）
tecsets_mdb

In [7]:
#各データのデータ型を整理
tecsets_mdb = tecsets_mdb.astype({'外径\nDia.':'float',\
                                  'コーナ\n半径\nCorner Radius':'float',\
                                  '首下長\nUnder Neck Length':'float',\
                                  '材質':'str',\
                                  'min-1':'int',\
                                  'mm/min':'int',\
                                  'ap mm':'float',\
                                  'ae mm':'float'})

#加工条件ファイル（完成版）
pd.set_option('display.max_rows',None)#行制限無し
tecsets_mdb

###################
ここから形状データ編集
###################

In [8]:
#カラムリスト
excel_columns = ('工具名称','工具コメント','工具直径','工具全長','有効刃長','コーナーR','刃数','シャンク定義','シャンク直径','先端長さ','シャンク角度','シャンク長さ','シャンク長さ（絶対値）','ネック有無','ネック直径','テーパー工具','テーパー工具角度','中心直径','中心高さ','回転方向')
eXcel_tecsets_colums = ('S:主軸回転速度','スピンドル回転数計算式','Vc:切削速度','FX:XY送り速度','XY送り速度計算式','FZ:Z送り速度','送り速度Z計算式','F/刃数(fz)','減速送り速度','減速送り速度計算式','クーラント','溝削り送り速度','Fz穴あけ(f)','切削幅(ae)','切削深さ(ap)','減速送り速度の最大角度','プランジ角度')

In [9]:
#スクレイピング段階での要素リスト(リネーム用のリストと順番を合わせる)
data_columns = ('呼び記号','DC','LF','APMX','RE','DCON','LU','DN')
#リネーム用のリスト
rename_columns = ('工具名称','工具直径','工具全長','有効刃長','コーナーR','シャンク直径','先端長さ','ネック直径')

In [10]:
for i in range(0,100,20):
    url = 'http://www.mitsubishicarbide.net/webcatalog/OMB04F001BLogic.do?srs_id=10000352&mkt_rykshu=mmc&gng_rykshu=ja&ctgr_rykshu=solid_end_mills&ngs_tni=M&hskzi_ini=&inst_ybkgu=&inst_zish_mi=&row=20&startIndex='+str(i)
    data = pd.read_html(url,encoding='UTF-8') #データを取得
    dfa = pd.merge(data[1],data[2], left_index=True, right_index=True) #ページ内のデータフレームを結合
    dfa = dfa[['呼び記号','図','DC','LF','APMX','RE','DCON','LU','DN']] #必要カラムのみにする。
    if i==0: #初回のみデータフレームを転送
        df=dfa
    else:
        df=df.append(dfa, ignore_index=True) #新しいデータフレームを下に結合       


In [11]:
#カラム名変更用の辞書を作成
rename_dic = dict(zip(data_columns, rename_columns)) 
#カラム名変更
df = df.rename(columns=rename_dic)

In [12]:
#EXCELフォーム用にカラムを追加
df = df.reindex(columns=excel_columns)

In [13]:
#工具名称入力
df = df.astype({'工具コメント':'object'}) #型変換
for i in range(0,len(df)):
    df.iat[i,1] = 'インパクトミラクル高能率加工用制振ラジアスエンドミル'

In [14]:
#決まっている値を入力
df['刃数'] = 4
df['テーパー工具'] = 0
df['中心直径'] = 0
df['中心高さ'] = 0
df['回転方向'] = 'clockwise'

In [15]:
#シャンク定義を条件に応じて値を修正
df = df.astype({'シャンク定義':'str','工具直径':'float','シャンク直径':'float'})
for i in range(0,len(df)):
    if df.at[i,'工具直径'] == df.at[i,'シャンク直径']:#ストレート工具を定義
        df.at[i,'ネック有無'] = 1
        df.at[i,'シャンク定義'] = 'length'
        df.at[i,'シャンク長さ'] = 0
    elif df.at[i,'工具直径'] > df.at[i,'シャンク直径']:
        df.at[i,'ネック有無'] = 1
        df.at[i,'シャンク定義'] = 'length'
        df.at[i,'シャンク長さ'] = 0
    else:
        df.at[i,'ネック有無'] = 1
        df.at[i,'シャンク定義'] = 'angle'#以外はネックテーパーにする
        df.at[i,'シャンク角度'] = 15


In [16]:
#各データのデータ型を整理
df = df.astype({'ネック有無':'int8'})

In [17]:
#NaN値を空白にする
df = df.replace(np.nan, '', regex=True)#regexオプションの意味が分からない

#テスト出力
pd.set_option('display.max_rows',None)
df

####################
ここから　XML定義
####################

In [18]:
#加工条件を材質ごとに書き込み
def write_tecset (temp_mat,temp_tecsets):
    tecsets = et.SubElement(tool, 'tecsets')
    for i in temp_mat:
        idnum = temp_tecsets.query('材質 == @i').index[0]
        
        tecset = et.SubElement(tecsets, 'tecset',{"type" : "milling"})
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("material")},value=str(i))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("cuttingMaterial")},value=str('超硬'))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("purpose")},value=str('multi'))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("lengthOfUnit")},value=str('mm'))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("spindleSpeed")},value=str(temp_tecsets.at[idnum,'min-1']))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("planeFeedrate")},value=str(temp_tecsets.at[idnum,'mm/min']))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("cuttingSpeed")},value=str('0'))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("coolants")},value=str('1'))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("cuttingDirection")},value=str("upAndDown"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("zFeedrate")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("feedratePerEdge")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("fullcutFeedrate")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("drillingFeedrate")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("cuttingWidth")},value=str(temp_tecsets.at[idnum,'ae mm']))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("cuttingLength")},value=str(temp_tecsets.at[idnum,'ap mm']))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("maxRedFeedrateAngle")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("plungeAngle")},value=str("2"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("reducedFeedrate")},value=str("0"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("planeFeedrateFormula")},value=str(""))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("zFeedrateFormula")},value=str("f送り速度Z"))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("spindleSpeedFormula")},value=str(""))
        add_tecsets = et.SubElement(tecset,'param', attrib={'name':str("reducedFeedrateFormula")},value=str("f減速"))

In [19]:
#各種読み込み用のカラムリストと出力用のカラムリストを定義
tecsets_list_all=['S:主軸回転速度','FX:XY送り速度','Vc:切削速度','クーラント','FZ:Z送り速度','減速送り速度','F/刃数(fz)','溝削り送り速度','Fz穴あけ(f)','切削幅(ae)','切削深さ(ap)','減速送り速度の最大角度','プランジ角度','XY送り速度計算式','送り速度Z計算式','スピンドル回転数計算式','減速送り速度計算式']
tecsets_param_list=['cuttingSpeed','coolants','feedratePerEdge','fullcutFeedrate','drillingFeedrate','cuttingWidth','cuttingLength','maxRedFeedrateAngle','plungeAngle']

In [20]:
#xmlヘッダー作成
omtdx = et.Element('omtdx', {"version" : "21" , "srcURL" : "" , "srcGuid" : "" , "srcType" : "standard" , "files" : ""})
tree = et.ElementTree(element=omtdx)
tools_a = et.SubElement(omtdx,'tools')

In [21]:
#xml作成用のリスト読み込み（今回はラジアス用）
shape_param_list=['radiusMill','comment','spindleRotation','toolTotalLength','cuttingEdges','cuttingLength','toolShaftDiameter','toolShaftChamferDefMode','toolShaftChamferLength','toolShaftChamferAbsPos','toolShaftChamferAngle','toolDiameter','taperHeight','collar','tipDiameter','tapered','taperAngle','cornerRadius','coreDiameter','coreHeight']
shape_list=['工具コメント','回転方向','工具全長','刃数','有効刃長','シャンク直径','シャンク定義','シャンク長さ','シャンク長さ（絶対値）','シャンク角度','工具直径','先端長さ','ネック有無','ネック直径','テーパー工具','テーパー工具角度','コーナーR','中心直径','中心高さ']

In [22]:
shape_param_list[4:6]

['cuttingEdges', 'cuttingLength']

In [23]:
#工具タイプ毎のxml作成

tools_b = et.SubElement(tools_a,'tools',{"folder" : folder_name , "comment" : "" , "objGuid" : ""})
for i in range(len(df)):

    #形状リストから行の値を取得してリスト化
    shape_value_list=[]
    for a in shape_list:
        b=(df.iloc[i][a])
        shape_value_list.append(b)
        #print (shape_value_list)

    #工具データ入力開始
    tool = et.SubElement(tools_b, 'tool',{"type" : shape_param_list[0]})

    #第一形状データの入力
    tool.set("name",str(df.iloc[i]['工具名称']))#単独で値を取得
    
    #第一形状連続入力
    add_shape = et.SubElement(tool,'param', attrib={'name':str(shape_param_list[1])},value=str(shape_value_list[0]))
    add_shape = et.SubElement(tool,'param', attrib={'name':str("objGuid")},value=str(""))
    add_shape = et.SubElement(tool,'param', attrib={'name':str("cuttingMaterial")},value=str("超硬"))
    add_shape = et.SubElement(tool,'param', attrib={'name':str("lengthOfUnit")},value=str("mm"))  
    
    
    for (palam_name,value_name) in zip(shape_param_list[2:4],shape_value_list[1:3]):
        add_shape = et.SubElement(tool,'param', attrib={'name':str(palam_name)},value=str(value_name))
    
    coupling = et.SubElement(tool, 'coupling',{"type" : "unknown" , "location" : "top"})
    add_shape = et.SubElement(coupling,'param', attrib={'name':str("objGuid")},value=str(""))
        
    for (palam_name,value_name) in zip(shape_param_list[4:6],shape_value_list[3:5]):
        add_shape = et.SubElement(tool,'param', attrib={'name':str(palam_name)},value=str(value_name))

    
    
    if df.at[i,'シャンク定義'] == 'straight':
        add_shape = et.SubElement(tool,'param', attrib={'name':'toolShaftType'},value="none")
        for (palam_name,value_name) in zip(shape_param_list[11:13],shape_value_list[10:12]):
            add_shape = et.SubElement(tool,'param', attrib={'name':str(palam_name)},value=str(value_name))
    else:
        add_shape = et.SubElement(tool,'param', attrib={'name':'toolShaftType'},value="parametric")
        for (palam_name,value_name) in zip(shape_param_list[6:13],shape_value_list[5:12]):
            add_shape = et.SubElement(tool,'param', attrib={'name':str(palam_name)},value=str(value_name))
    
    
    #切削条件データの入力
    #データ抽出 工具サイズから必要な条件を抽出
    dia = df.at[i,'工具直径']
    nose_r = df.at[i,'コーナーR']
    neck = df.at[i,'先端長さ']

    if not neck:
        neck = 0
    
    temp_tecsets = tecsets_mdb[(tecsets_mdb['外径\nDia.'] == dia) & (tecsets_mdb['コーナ\n半径\nCorner Radius'] == nose_r) & (tecsets_mdb['首下長\nUnder Neck Length'] == neck) ]
    temp_tecsets = temp_tecsets.reset_index(drop=True)  #再インデックス
    temp_mat = temp_tecsets['材質']
    
    #加工条件入力関数呼び出し
    write_tecset (temp_mat,temp_tecsets)    


    #第二形状データの入力
    for (palam_name,value_name) in zip(shape_param_list[13:],shape_value_list[12:]):
        add_shape = et.SubElement(tool,'param', attrib={'name':str(palam_name)},value=str(value_name))


In [24]:
tree.write(export_file_name, encoding='utf-8', xml_declaration=True) #ファイル出力

In [23]:
tree.write(export_file_name, encoding='utf-8', xml_declaration=True) #ファイル出力